In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.layers import Input, Dense, Flatten, Dropout
from tensorflow.keras import Model

In [11]:
df = pd.read_csv('./data/clean_data_with_date_weather.csv', index_col=0)
df

,index,county,state,season,title,latitude,longitude,date,number,geohash,...,dew_point,humidity,cloud_cover,precip_type,pressure,wind_bearing,wind_speed,Class A,Class B,Class C
0,2,Washington County,Rhode Island,Fall,Report 6496: Bicycling student has night encou...,41.45000,-71.50000,1974-09-20,6496.0,drm5ucxrc0,...,66.227150,0.907111,0.268375,rain,1019.476257,263.869568,11.258579,1,0,0
1,5,Washita County,Oklahoma,Fall,Report 9765: Motorist and children have daylig...,35.30110,-99.17020,1973-09-28,9765.0,9y32z667yc,...,47.139355,0.646660,0.113750,No Precipitation,953.307434,164.583038,14.922651,1,0,0
2,6,Washington County,Ohio,Summer,Report 4983: Sighting and vocalizations at a h...,39.38745,-81.67339,1971-08-01,4983.0,dpjbj6r280,...,60.795803,0.745234,0.584542,No Precipitation,993.191711,17.920425,4.966946,1,0,0
3,7,Westchester County,New York,Fall,Report 31940: Fishermen and Law Enforcement Of...,41.29873,-73.73717,2010-09-01,31940.0,dr7dfn4nx3,...,63.793255,0.578789,0.126625,No Precipitation,1001.331787,270.616058,6.397976,1,0,0
4,8,Washoe County,Nevada,Fall,Report 5692: Father and son hear vocalizations...,39.59200,-119.97500,1970-09-01,5692.0,9r49jzsb5g,...,29.029099,0.402105,0.000000,No Precipitation,807.823975,219.053116,9.812248,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4034,5010,Harlan County,Kentucky,Fall,Report 25257: ATV riders find possible tracks ...,36.89771,-83.16293,2008-10-15,25257.0,dnscgt48er,...,54.413593,0.740113,0.151125,No Precipitation,911.663757,201.991241,5.410581,0,1,0
4035,5011,Harlan County,Kentucky,Winter,Report 38980: Teen on four-wheeler has sightin...,36.83193,-83.42429,2013-01-10,38980.0,dns9t9h391,...,47.541798,0.930158,0.878042,rain,980.778503,89.679916,6.219504,1,0,0
4036,5012,Anderson County,Kentucky,Fall,Report 1045: Father and son have a daylight en...,37.95708,-84.97527,2005-01-09,1045.0,dnexfmfuh8,...,33.796551,0.793257,0.548708,No Precipitation,995.131042,178.840332,7.880466,1,0,0
4037,5013,Anderson County,Kentucky,Fall,Report 33187: Dual sighting in late afternoon ...,37.97699,-84.90234,2011-11-15,33187.0,dng8h18000,...,56.680698,0.936365,0.963083,rain,984.935791,201.164108,13.016956,1,0,0


In [12]:
def one_hot_encode(data, col_name):
    data = pd.concat([data, pd.get_dummies(data[col_name], dtype=int)], axis=1)
    data.drop(col_name, axis=1, inplace=True)
    return data

In [13]:
season_dict = {
    '01': 'Winter',
    '02': 'Winter',
    '03': 'Spring',
    '04': 'Spring',
    '05': 'Spring',
    '06': 'Summer',
    '07': 'Summer',
    '08': 'Summer',
    '09': 'Fall',
    '10': 'Fall',
    '11': 'Fall',
    '12': 'Winter',
}

season_col = df.columns.get_loc('season')
for row_idx in df[df['season'] == 'Unknown'].index:
    month = df.iloc[row_idx, :]['date'].split('-')[1]
    df.iloc[row_idx, season_col] = season_dict[month]

In [14]:
# Convert to unix timestamp
df['date'] = df['date'].astype(np.datetime64).astype('int64')/1e9

In [18]:
x_df = df.drop(['index', 'county', 'state', 'title', 'geohash', 'number', 'Class A', 'Class B', 'Class C'], axis=1)
x_df = one_hot_encode(x_df, 'season')
x_df = one_hot_encode(x_df, 'precip_type')
x_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4039 entries, 0 to 4038
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   latitude          4039 non-null   float64
 1   longitude         4039 non-null   float64
 2   date              4039 non-null   float64
 3   temperature_high  4039 non-null   float64
 4   temperature_mid   4039 non-null   float64
 5   temperature_low   4039 non-null   float64
 6   dew_point         4039 non-null   float64
 7   humidity          4039 non-null   float64
 8   cloud_cover       4039 non-null   float64
 9   pressure          4039 non-null   float64
 10  wind_bearing      4039 non-null   float64
 11  wind_speed        4039 non-null   float64
 12  Fall              4039 non-null   int32  
 13  Spring            4039 non-null   int32  
 14  Summer            4039 non-null   int32  
 15  Winter            4039 non-null   int32  
 16  No Precipitation  4039 non-null   int32  


In [57]:
x_train, x_test = train_test_split(x_df, test_size = 0.2)

latitude_train = np.array(x_train['latitude'])
longitude_train = np.array(x_train['longitude'])

latitude_test = np.array(x_test['latitude'])
longitude_test = np.array(x_test['longitude'])

x_train = x_train.drop(['latitude', 'longitude'], axis=1)
x_test = x_test.drop(['latitude', 'longitude'], axis=1)

# Normalize inputs
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [58]:
def create_model(input_len, optimizer=tf.keras.optimizers.SGD(learning_rate=0.001), loss_type='mse'):
    # https://manuel-gilm.medium.com/multi-target-regression-with-tensorflow-a7d974cf5d89
    input_layer = Input(shape=input_len)
    dense_layer_1 = Dense(units=128, activation="relu")(input_layer) 
    dense_layer_2 = Dense(units=128, activation="relu")(dense_layer_1)
    dense_layer_3 = Dense(units=64, activation="relu")(dense_layer_2)
    
    latitude_output = Dense(units=1, activation="linear", name="latitude_output")(dense_layer_2)
    longitude_output = Dense(units=1, activation="linear", name="longitude_output")(dense_layer_3)
    
    model = Model(inputs=input_layer, outputs=[latitude_output, longitude_output])
        
    model.compile(optimizer = optimizer,
                  loss = {'latitude_output': loss_type, 'longitude_output': loss_type},
                  metrics = {
                      'latitude_output': tf.keras.metrics.RootMeanSquaredError(),
                      'longitude_output': tf.keras.metrics.RootMeanSquaredError(),
                  }
                 )
    
    return model

In [59]:
x = np.array(x_train)
y = np.array([latitude_train, longitude_train])
x_test = np.array(x_test)
y_test = np.array([latitude_test, longitude_test])

In [55]:
model = create_model(x_train.shape[1], tf.keras.optimizers.Adam(learning_rate=0.001))
history = model.fit(x, y.T, epochs=100, batch_size=32,
                    validation_data = (x_test, y_test.T), verbose=1)

Epoch 1/100
101/101 [==============================] - 1s 6ms/step - loss: 10101.1689 - latitude_output_loss: 5060.1167 - longitude_output_loss: 5041.0513 - latitude_output_root_mean_squared_error: 71.1345 - longitude_output_root_mean_squared_error: 71.0004 - val_loss: 9651.7715 - val_latitude_output_loss: 4826.3242 - val_longitude_output_loss: 4825.4453 - val_latitude_output_root_mean_squared_error: 69.4717 - val_longitude_output_root_mean_squared_error: 69.4654
Epoch 2/100
101/101 [==============================] - 1s 5ms/step - loss: 9653.2207 - latitude_output_loss: 4826.5366 - longitude_output_loss: 4826.6826 - latitude_output_root_mean_squared_error: 69.4733 - longitude_output_root_mean_squared_error: 69.4743 - val_loss: 9635.0459 - val_latitude_output_loss: 4817.5762 - val_longitude_output_loss: 4817.4697 - val_latitude_output_root_mean_squared_error: 69.4088 - val_longitude_output_root_mean_squared_error: 69.4080
Epoch 3/100
101/101 [==============================] - 0s 5ms/ste

In [48]:
model = create_model(x_train.shape[1], tf.keras.optimizers.Adam(learning_rate=0.001), loss_type='mae')
history = model.fit(x, y.T, epochs=100, batch_size=32,
                    validation_data = (x_test, y_test.T), verbose=1)

Epoch 1/100
101/101 [==============================] - 1s 7ms/step - loss: 136.6396 - latitude_output_loss: 68.3198 - longitude_output_loss: 68.3198 - latitude_output_mean_absolute_error: 68.3198 - longitude_output_mean_absolute_error: 68.3198 - val_loss: 137.4212 - val_latitude_output_loss: 68.7106 - val_longitude_output_loss: 68.7106 - val_latitude_output_mean_absolute_error: 68.7106 - val_longitude_output_mean_absolute_error: 68.7106
Epoch 2/100
101/101 [==============================] - 1s 5ms/step - loss: 136.6395 - latitude_output_loss: 68.3198 - longitude_output_loss: 68.3198 - latitude_output_mean_absolute_error: 68.3198 - longitude_output_mean_absolute_error: 68.3198 - val_loss: 137.4212 - val_latitude_output_loss: 68.7106 - val_longitude_output_loss: 68.7106 - val_latitude_output_mean_absolute_error: 68.7106 - val_longitude_output_mean_absolute_error: 68.7106
Epoch 3/100
101/101 [==============================] - 0s 4ms/step - loss: 136.6395 - latitude_output_loss: 68.3198 - 

In [49]:
model = create_model(x_train.shape[1], loss_type='mae')
history = model.fit(x, y.T, epochs=100, batch_size=32,
                    validation_data = (x_test, y_test.T), verbose=1)

Epoch 1/100
101/101 [==============================] - 1s 5ms/step - loss: 136.6396 - latitude_output_loss: 68.3198 - longitude_output_loss: 68.3198 - latitude_output_mean_absolute_error: 68.3198 - longitude_output_mean_absolute_error: 68.3198 - val_loss: 137.4212 - val_latitude_output_loss: 68.7106 - val_longitude_output_loss: 68.7106 - val_latitude_output_mean_absolute_error: 68.7106 - val_longitude_output_mean_absolute_error: 68.7106
Epoch 2/100
101/101 [==============================] - 0s 5ms/step - loss: 136.6395 - latitude_output_loss: 68.3198 - longitude_output_loss: 68.3198 - latitude_output_mean_absolute_error: 68.3198 - longitude_output_mean_absolute_error: 68.3198 - val_loss: 137.4212 - val_latitude_output_loss: 68.7106 - val_longitude_output_loss: 68.7106 - val_latitude_output_mean_absolute_error: 68.7106 - val_longitude_output_mean_absolute_error: 68.7106
Epoch 3/100
101/101 [==============================] - 0s 4ms/step - loss: 136.6395 - latitude_output_loss: 68.3198 - 

In [41]:
model = create_model(x_train.shape[1], tf.keras.optimizers.SGD(learning_rate=0.001))
history = model.fit(x, y.T, epochs=100, batch_size=32,
                    validation_data = (x_test, y_test.T), verbose=1)

Epoch 1/100
101/101 [==============================] - 1s 7ms/step - loss: 9740.1914 - latitude_output_loss: 4875.5161 - longitude_output_loss: 4864.6748 - latitude_output_root_mean_squared_error: 69.8249 - longitude_output_root_mean_squared_error: 69.7472 - val_loss: 9728.1982 - val_latitude_output_loss: 4863.7485 - val_longitude_output_loss: 4864.4487 - val_latitude_output_root_mean_squared_error: 69.7406 - val_longitude_output_root_mean_squared_error: 69.7456
Epoch 2/100
101/101 [==============================] - 0s 5ms/step - loss: 9619.9775 - latitude_output_loss: 4807.9990 - longitude_output_loss: 4811.9766 - latitude_output_root_mean_squared_error: 69.3397 - longitude_output_root_mean_squared_error: 69.3684 - val_loss: 9726.1035 - val_latitude_output_loss: 4862.8467 - val_longitude_output_loss: 4863.2573 - val_latitude_output_root_mean_squared_error: 69.7341 - val_longitude_output_root_mean_squared_error: 69.7371
Epoch 3/100
101/101 [==============================] - 0s 4ms/step

In [19]:
# https://machinelearningmastery.com/deep-learning-models-for-multi-output-regression/
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
 

def get_dataset():
    x = x_df.drop(['latitude', 'longitude'], axis=1)
                  
    scaler = StandardScaler()
    scaler.fit(x)
    x = scaler.transform(x)
    
    y = x_df[['longitude', 'latitude']]
    
    return x, y
 
# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y, model_fn, epochs=100, repeats=3):
    results = list()
    n_inputs, n_outputs = X.shape[1], y.shape[1]
    
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=repeats, random_state=1)
    # enumerate folds
    for train_ix, test_ix in cv.split(X):
        # prepare data
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y.iloc[train_ix, :], y.iloc[test_ix, :]
        # define model
        model = model_fn(n_inputs, n_outputs)
        # fit model
        model.fit(X_train, y_train, verbose=1, epochs=epochs)
        # evaluate model on test set
        mae = model.evaluate(X_test, y_test, verbose=1)
        
        # store result
        print('>%.3f' % mae)
        results.append(mae)
    return results
 
x, y = get_dataset()

In [11]:
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs))
    model.compile(loss='mae', optimizer='adam')
    return model

# evaluate model
results = evaluate_model(x, y, get_model, epochs=100, repeats=1)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(results), std(results)))

Epoch 1/100
114/114 [==============================] - 1s 3ms/step - loss: 66.4181
Epoch 2/100
114/114 [==============================] - 0s 3ms/step - loss: 62.7357
Epoch 3/100
114/114 [==============================] - 0s 2ms/step - loss: 56.4818
Epoch 4/100
114/114 [==============================] - 0s 2ms/step - loss: 47.6926
Epoch 5/100
114/114 [==============================] - 0s 2ms/step - loss: 38.7531
Epoch 6/100
114/114 [==============================] - 0s 2ms/step - loss: 31.1056
Epoch 7/100
114/114 [==============================] - 0s 2ms/step - loss: 24.6571
Epoch 8/100
114/114 [==============================] - 0s 2ms/step - loss: 19.6649
Epoch 9/100
114/114 [==============================] - 0s 2ms/step - loss: 16.1577
Epoch 10/100
114/114 [==============================] - 0s 2ms/step - loss: 14.1913
Epoch 11/100
114/114 [==============================] - 0s 2ms/step - loss: 13.1251
Epoch 12/100
114/114 [==============================] - 0s 2ms/step - loss: 12.4418
E

In [12]:
def get_model_2(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(n_outputs))
    model.compile(loss='mae', optimizer='adam')
    return model

# evaluate model
results = evaluate_model(x, y, get_model_2, epochs=100, repeats=1)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(results), std(results)))

Epoch 1/100
114/114 [==============================] - 1s 4ms/step - loss: 67.3331
Epoch 2/100
114/114 [==============================] - 0s 2ms/step - loss: 57.5941
Epoch 3/100
114/114 [==============================] - 0s 4ms/step - loss: 38.1854
Epoch 4/100
114/114 [==============================] - 0s 3ms/step - loss: 19.0014
Epoch 5/100
114/114 [==============================] - 0s 3ms/step - loss: 12.3187
Epoch 6/100
114/114 [==============================] - 0s 2ms/step - loss: 10.8205
Epoch 7/100
114/114 [==============================] - 0s 3ms/step - loss: 10.1422
Epoch 8/100
114/114 [==============================] - 0s 3ms/step - loss: 9.6610
Epoch 9/100
114/114 [==============================] - 0s 2ms/step - loss: 9.2949
Epoch 10/100
114/114 [==============================] - 0s 3ms/step - loss: 8.9807
Epoch 11/100
114/114 [==============================] - 0s 3ms/step - loss: 8.7199
Epoch 12/100
114/114 [==============================] - 0s 3ms/step - loss: 8.5077
Epoch 

In [13]:
def get_model_3(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(128, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(n_outputs))
    model.compile(loss='mae', optimizer='adam')
    return model

# evaluate model
results = evaluate_model(x, y, get_model_3, epochs=100, repeats=1)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(results), std(results)))

Epoch 1/100
114/114 [==============================] - 1s 3ms/step - loss: 60.1768
Epoch 2/100
114/114 [==============================] - 0s 3ms/step - loss: 32.8537
Epoch 3/100
114/114 [==============================] - 0s 2ms/step - loss: 11.2856
Epoch 4/100
114/114 [==============================] - 0s 2ms/step - loss: 9.1975
Epoch 5/100
114/114 [==============================] - 0s 2ms/step - loss: 8.6137
Epoch 6/100
114/114 [==============================] - 0s 2ms/step - loss: 8.2761
Epoch 7/100
114/114 [==============================] - 0s 2ms/step - loss: 8.0346
Epoch 8/100
114/114 [==============================] - 0s 2ms/step - loss: 7.8311
Epoch 9/100
114/114 [==============================] - 0s 2ms/step - loss: 7.6883
Epoch 10/100
114/114 [==============================] - 0s 3ms/step - loss: 7.5473
Epoch 11/100
114/114 [==============================] - 0s 2ms/step - loss: 7.4694
Epoch 12/100
114/114 [==============================] - 0s 2ms/step - loss: 7.3809
Epoch 13/1

In [14]:
def get_model_4(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(128, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(n_outputs))
    model.compile(loss='mae', optimizer='adam')
    return model

# evaluate model
results = evaluate_model(x, y, get_model_4, epochs=100, repeats=1)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(results), std(results)))

Epoch 1/100
114/114 [==============================] - 1s 3ms/step - loss: 38.9625
Epoch 2/100
114/114 [==============================] - 0s 3ms/step - loss: 10.2147
Epoch 3/100
114/114 [==============================] - 0s 2ms/step - loss: 8.8396
Epoch 4/100
114/114 [==============================] - 0s 2ms/step - loss: 8.2674
Epoch 5/100
114/114 [==============================] - 0s 2ms/step - loss: 7.9477
Epoch 6/100
114/114 [==============================] - 0s 3ms/step - loss: 7.7401
Epoch 7/100
114/114 [==============================] - 0s 2ms/step - loss: 7.5217
Epoch 8/100
114/114 [==============================] - 0s 2ms/step - loss: 7.4091
Epoch 9/100
114/114 [==============================] - 0s 2ms/step - loss: 7.2846
Epoch 10/100
114/114 [==============================] - 0s 2ms/step - loss: 7.1342
Epoch 11/100
114/114 [==============================] - 0s 2ms/step - loss: 7.0383
Epoch 12/100
114/114 [==============================] - 0s 2ms/step - loss: 6.9804
Epoch 13/10

In [15]:
def get_model_5(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(256, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(n_outputs))
    model.compile(loss='mae', optimizer='adam')
    return model

# evaluate model
results = evaluate_model(x, y, get_model_5, epochs=100, repeats=1)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(results), std(results)))

Epoch 1/100
114/114 [==============================] - 1s 4ms/step - loss: 24.2047
Epoch 2/100
114/114 [==============================] - 0s 3ms/step - loss: 8.7765
Epoch 3/100
114/114 [==============================] - 0s 3ms/step - loss: 7.8622
Epoch 4/100
114/114 [==============================] - 0s 3ms/step - loss: 7.5164
Epoch 5/100
114/114 [==============================] - 0s 3ms/step - loss: 7.2405
Epoch 6/100
114/114 [==============================] - 0s 3ms/step - loss: 7.0483
Epoch 7/100
114/114 [==============================] - 0s 3ms/step - loss: 6.8556
Epoch 8/100
114/114 [==============================] - 0s 3ms/step - loss: 6.7096
Epoch 9/100
114/114 [==============================] - 0s 2ms/step - loss: 6.6313
Epoch 10/100
114/114 [==============================] - 0s 3ms/step - loss: 6.5480
Epoch 11/100
114/114 [==============================] - 0s 2ms/step - loss: 6.4087
Epoch 12/100
114/114 [==============================] - 0s 3ms/step - loss: 6.4217
Epoch 13/100

In [16]:
def get_model_6(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(128, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(n_outputs))
    model.compile(loss='mae', optimizer='adam')
    return model

# evaluate model
results = evaluate_model(x, y, get_model_6, epochs=100, repeats=1)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(results), std(results)))

Epoch 1/100
114/114 [==============================] - 1s 3ms/step - loss: 35.9005
Epoch 2/100
114/114 [==============================] - 0s 2ms/step - loss: 9.5996
Epoch 3/100
114/114 [==============================] - 0s 3ms/step - loss: 8.5393
Epoch 4/100
114/114 [==============================] - 0s 2ms/step - loss: 7.9724
Epoch 5/100
114/114 [==============================] - 0s 2ms/step - loss: 7.6350
Epoch 6/100
114/114 [==============================] - 0s 2ms/step - loss: 7.3925
Epoch 7/100
114/114 [==============================] - 0s 2ms/step - loss: 7.2450
Epoch 8/100
114/114 [==============================] - 0s 2ms/step - loss: 7.0778
Epoch 9/100
114/114 [==============================] - 0s 2ms/step - loss: 6.9428
Epoch 10/100
114/114 [==============================] - 0s 2ms/step - loss: 6.8739
Epoch 11/100
114/114 [==============================] - 0s 2ms/step - loss: 6.7181
Epoch 12/100
114/114 [==============================] - 0s 3ms/step - loss: 6.6398
Epoch 13/100

In [9]:
def get_model_7(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(256, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(n_outputs))
    model.compile(loss='mae', optimizer='adam')
    return model

# evaluate model
results = evaluate_model(x, y, get_model_7, epochs=100, repeats=1)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(results), std(results)))

Epoch 1/100
114/114 [==============================] - 1s 3ms/step - loss: 22.9930
Epoch 2/100
114/114 [==============================] - 0s 3ms/step - loss: 9.0405
Epoch 3/100
114/114 [==============================] - 0s 3ms/step - loss: 8.0668
Epoch 4/100
114/114 [==============================] - 0s 3ms/step - loss: 7.4986
Epoch 5/100
114/114 [==============================] - 0s 3ms/step - loss: 7.2107
Epoch 6/100
114/114 [==============================] - 0s 3ms/step - loss: 7.0211
Epoch 7/100
114/114 [==============================] - 0s 3ms/step - loss: 6.8086
Epoch 8/100
114/114 [==============================] - 0s 3ms/step - loss: 6.7655
Epoch 9/100
114/114 [==============================] - 0s 3ms/step - loss: 6.4306
Epoch 10/100
114/114 [==============================] - 0s 3ms/step - loss: 6.3220
Epoch 11/100
114/114 [==============================] - 0s 3ms/step - loss: 6.2060
Epoch 12/100
114/114 [==============================] - 0s 3ms/step - loss: 6.0628
Epoch 13/100

In [22]:
def get_model_8(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(256, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(n_outputs))
    model.compile(loss='mae', optimizer='adam')
    return model

# evaluate model
results = evaluate_model(x, y, get_model_8, epochs=100, repeats=1)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(results), std(results)))

# Adding Dropout layers did not help (tried in between each hidden layer)

Epoch 1/100
114/114 [==============================] - 1s 3ms/step - loss: 26.1172
Epoch 2/100
114/114 [==============================] - 0s 3ms/step - loss: 13.2721
Epoch 3/100
114/114 [==============================] - 0s 3ms/step - loss: 12.0337
Epoch 4/100
114/114 [==============================] - 0s 3ms/step - loss: 11.2902
Epoch 5/100
114/114 [==============================] - 0s 3ms/step - loss: 10.9323
Epoch 6/100
114/114 [==============================] - 0s 3ms/step - loss: 10.9761
Epoch 7/100
114/114 [==============================] - 0s 3ms/step - loss: 10.4236
Epoch 8/100
114/114 [==============================] - 0s 3ms/step - loss: 10.2676
Epoch 9/100
114/114 [==============================] - 0s 3ms/step - loss: 10.3803
Epoch 10/100
114/114 [==============================] - 0s 3ms/step - loss: 10.2505
Epoch 11/100
114/114 [==============================] - 0s 3ms/step - loss: 10.0087
Epoch 12/100
114/114 [==============================] - 0s 3ms/step - loss: 9.9101
Ep

In [10]:
def get_model_9(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(256, input_dim=n_inputs, kernel_initializer='lecun_uniform', activation='selu'))
    model.add(Dense(128, activation='selu'))
    model.add(Dense(64, activation='selu'))
    model.add(Dense(32, activation='selu'))
    model.add(Dense(n_outputs))
    model.compile(loss='mae', optimizer='adam')
    return model

# evaluate model
results = evaluate_model(x, y, get_model_9, epochs=100, repeats=1)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(results), std(results)))

Epoch 1/100
114/114 [==============================] - 1s 3ms/step - loss: 34.7003
Epoch 2/100
114/114 [==============================] - 0s 2ms/step - loss: 8.4166
Epoch 3/100
114/114 [==============================] - 0s 3ms/step - loss: 7.3209
Epoch 4/100
114/114 [==============================] - 0s 3ms/step - loss: 6.9620
Epoch 5/100
114/114 [==============================] - 0s 3ms/step - loss: 6.8797
Epoch 6/100
114/114 [==============================] - 0s 3ms/step - loss: 6.7566
Epoch 7/100
114/114 [==============================] - 0s 2ms/step - loss: 6.4885
Epoch 8/100
114/114 [==============================] - 0s 2ms/step - loss: 6.3365
Epoch 9/100
114/114 [==============================] - 0s 3ms/step - loss: 6.3616
Epoch 10/100
114/114 [==============================] - 0s 2ms/step - loss: 6.2494
Epoch 11/100
114/114 [==============================] - 0s 2ms/step - loss: 6.1549
Epoch 12/100
114/114 [==============================] - 0s 2ms/step - loss: 6.1105
Epoch 13/100

In [11]:
results_1 = evaluate_model(x, y, get_model_7, epochs=250, repeats=1)
results_2 = evaluate_model(x, y, get_model_9, epochs=250, repeats=1)

print('MAE: %.3f (%.3f)' % (mean(results_1), std(results_1)))
print('MAE: %.3f (%.3f)' % (mean(results_2), std(results_2)))

Epoch 1/250
114/114 [==============================] - 1s 3ms/step - loss: 26.3242
Epoch 2/250
114/114 [==============================] - 0s 3ms/step - loss: 9.3573
Epoch 3/250
114/114 [==============================] - 0s 2ms/step - loss: 8.2299
Epoch 4/250
114/114 [==============================] - 0s 2ms/step - loss: 7.7433
Epoch 5/250
114/114 [==============================] - 0s 2ms/step - loss: 7.3082
Epoch 6/250
114/114 [==============================] - 0s 2ms/step - loss: 7.1703
Epoch 7/250
114/114 [==============================] - 0s 2ms/step - loss: 6.9089
Epoch 8/250
114/114 [==============================] - 0s 2ms/step - loss: 6.7465
Epoch 9/250
114/114 [==============================] - 0s 2ms/step - loss: 6.5456
Epoch 10/250
114/114 [==============================] - 0s 2ms/step - loss: 6.5137
Epoch 11/250
114/114 [==============================] - 0s 2ms/step - loss: 6.3546
Epoch 12/250
114/114 [==============================] - 0s 2ms/step - loss: 6.2880
Epoch 13/250

In [26]:
model = get_model_7(x.shape[1], y.shape[1])
model.fit(x, y, verbose=1, epochs=5_000)

Epoch 1/5000
127/127 [==============================] - 1s 4ms/step - loss: 22.0826
Epoch 2/5000
127/127 [==============================] - 0s 3ms/step - loss: 9.1474
Epoch 3/5000
127/127 [==============================] - 0s 3ms/step - loss: 7.9464
Epoch 4/5000
127/127 [==============================] - 0s 3ms/step - loss: 7.3846
Epoch 5/5000
127/127 [==============================] - 0s 3ms/step - loss: 6.9108
Epoch 6/5000
127/127 [==============================] - 0s 3ms/step - loss: 6.8165
Epoch 7/5000
127/127 [==============================] - 0s 3ms/step - loss: 6.6090
Epoch 8/5000
127/127 [==============================] - 0s 3ms/step - loss: 6.4505
Epoch 9/5000
127/127 [==============================] - 0s 3ms/step - loss: 6.2793
Epoch 10/5000
127/127 [==============================] - 0s 3ms/step - loss: 6.2083
Epoch 11/5000
127/127 [==============================] - 0s 3ms/step - loss: 6.0631
Epoch 12/5000
127/127 [==============================] - 0s 3ms/step - loss: 6.0050


In [27]:
model.save('./models/final_dnn.keras')

In [28]:
model_loaded = tf.keras.models.load_model('./models/final_dnn.keras')
model_loaded

In [48]:
test_df = pd.read_csv('./data/test_input.csv', index_col=0)
test_df = one_hot_encode(test_df, 'precip_type')
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 0 to 0
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   latitude          1 non-null      float64
 1   longitude         1 non-null      float64
 2   date              1 non-null      object 
 3   temperature_high  1 non-null      float64
 4   temperature_mid   1 non-null      float64
 5   temperature_low   1 non-null      float64
 6   dew_point         1 non-null      float64
 7   humidity          1 non-null      float64
 8   cloud_cover       1 non-null      float64
 9   pressure          1 non-null      float64
 10  wind_bearing      1 non-null      float64
 11  wind_speed        1 non-null      float64
 12  Fall              1 non-null      int64  
 13  Spring            1 non-null      int64  
 14  Summer            1 non-null      int64  
 15  Winter            1 non-null      int64  
 16  No Precipitation  1 non-null      int64  
 17  r

In [62]:
test_df['date'] = test_df['date'].astype('datetime64[ns]').astype('int64')/1e9

x_test = test_df.drop(['latitude', 'longitude'], axis=1)
                  
scaler = StandardScaler()
scaler.fit(x_test)
x_test = scaler.transform(x_test)

pred = model.predict(x_test)
print(f'Predicted: {pred}')

1/1 [==============================] - 0s 84ms/step
Predicted: [[-41.801414  14.913077]]
